In [41]:
import pandas as pd
import sqlite3
orders = pd.read_csv("/content/orders.csv")
print("Orders shape:", orders.shape)
orders.head()

users = pd.read_json("/content/users.json")
print("Users shape:", users.shape)
users.head()

conn = sqlite3.connect("restaurants.db")
with open("/content/restaurants.sql", "r") as f:
    sql_script = f.read()
conn.executescript(sql_script)
print("restaurants.sql loaded successfully")

Orders shape: (10000, 6)
Users shape: (3000, 4)
restaurants.sql loaded successfully


In [42]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants shape:", restaurants.shape)
restaurants.head()

Restaurants shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [43]:
df = orders.merge(users, on="user_id", how="left")
final_df = df.merge(
    restaurants,
    on="restaurant_id",
    how="left",
    suffixes=("_order", "_master")
)

In [44]:
len(orders), len(final_df)
final_df.columns
final_df.isna().sum()
list(final_df.columns)
final_df = final_df.drop(columns=["restaurant_name_x"], errors="ignore")
final_df = final_df.rename(columns={"restaurant_name_y": "restaurant_name"})
print("Final columns:")
print(final_df.columns)


Final columns:
Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_order', 'name', 'city', 'membership',
       'restaurant_name_master', 'cuisine', 'rating'],
      dtype='object')


In [45]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [46]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
gold_city_revenue

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [47]:
cuisine_avg_order = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
cuisine_avg_order

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [48]:
user_total_spend = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)
(user_total_spend > 1000).sum()

np.int64(2544)

In [49]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]
)
final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

/tmp/ipython-input-2089426043.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6 – 5.0,2197030.75
4.1 – 4.5,1960326.26
3.0 – 3.5,1881754.57
3.6 – 4.0,1717494.41


In [50]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [51]:
(
    final_df
    .groupby("cuisine")
    .agg(
        restaurant_count=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(by=["restaurant_count", "total_revenue"], ascending=[True, False])
)

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [52]:
(
    final_df[final_df["membership"] == "Gold"].shape[0]
    / final_df.shape[0]
) * 100

49.87

In [53]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

(
    final_df[final_df["restaurant_name_order"].isin(options)]
    .groupby("restaurant_name_order")
    .agg(
        avg_order_value=("total_amount", "mean"),
        order_count=("order_id", "count")
    )
    .query("order_count < 20")
    .sort_values("avg_order_value", ascending=False)
    .head(1)
    .reset_index()
)

,restaurant_name_order,avg_order_value,order_count
0,Ruchi Foods Chinese,686.603158,19


In [54]:
(
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [55]:
(
    final_df
    .assign(
        quarter=pd.to_datetime(final_df["order_date"]).dt.to_period("Q")
    )
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

/tmp/ipython-input-2373803402.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  quarter=pd.to_datetime(final_df["order_date"]).dt.to_period("Q")


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [56]:
final_df[final_df["membership"] == "Gold"].shape[0]

4987

In [57]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)

1889367

In [58]:
final_df["user_id"].nunique()

2883

In [59]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [60]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [61]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

1337